# Imports

In [9]:
import pandas as pd
import numpy as np 
import matplotlib.pyplot as plt 
from scipy.stats import multivariate_normal

# Implemenet GMM

In [200]:
class GMM:
    def __init__(self, k, k_means=True):
        """
        Init method
        param k: number of clusters
        param k_means(bool): use k-means for means init or not 
        """
        self.k = k
        self.k_means = k_means
        self.means = []
        self.covs = []
        self.pis = []
        self.gammas = []
    
    
    def e_step(self,X):
        """
        Implement E-step(calculate gammas)
        use from scipy.stats import multivariate_normal
        """
        gammas = np.zeros((X.shape[0],self.k))
        for k in range(self.k):
            gammas[:,k] = self.pis[k] * multivariate_normal.pdf(X, self.means[k,:], self.covs[k],allow_singular=True)
        gammas = gammas /np.sum(gammas, axis=1)[:,np.newaxis]
        self.gammas = gammas
        
        print(self.gammas)
        
    
    def m_step(self,X):
        """
        M-step (update means,pis,covs)
        """
        covs = np.zeros((self.k, X.shape[1], X.shape[1]))
        
        nk = np.sum(self.gammas,axis=0)[:,np.newaxis]
        pis =np.mean(self.gammas, axis = 0).reshape(-1,1)
        means = np.zeros((self.k, X.shape[1]))
        for k in range(self.k):
            means[k,:] = np.sum(self.gammas[:,k].T @ X)
            covs[k,:, :] =(np.sum(self.gammas[:,k].T @ (X - means[k,:]) @ (X - means[k,:]).T))/( np.sum(self.gammas, axis = 0)[:,np.newaxis][k])
        means /= nk
        self.means = means
        self.covs = covs
        self.pis = pis
    
    
        
    def fit(self,X):
        """
        Main fit method.Need to
        1. Initialize means, pis, covs
        2. E-step (calculate gammas)
        3. M-step (update means,pis,covs)
        4. Repeat to converge(means dosent not change, or loss dosent not)
        """
        self.means = np.zeros((self.k,X.shape[1]))
        self.pis = np.zeros((self.k,1))+1/self.k
        identity = np.identity(X.shape[1])
        covs = np.zeros((self.k, X.shape[1], X.shape[1]))
        for k in range(self.k):
            covs[k,:, :] = identity
        self.covs = covs
        n=1
        inital_log = self.log_likelihood(X)
        temp = inital_log
        diff = float('inf')
        n_iter = 300
        while n!=n_iter:
            self.e_step(X)
            self.m_step(X)
            n+=1
            if(temp == self.log_likelihood(X)):
                n = n_iter
            else:
                temp = self.log_likelihood(X)
            
            
    
    def predict(self,X):
        """
        Calculate probabilites(gammas) for new X 
        param X(nd array): input dataset
        """
        y_probs = np.zeros((X.shape[0], self.k))
        for k in range(self.k):
            y_probs [:,k] = self.pis[k] * multivariate_normal.pdf(X, self.means[k,:], self.covs[k],allow_singular=True)
        return y_probs 
    
    def log_likelihood(self,X):
        """
        Calculate and return log-likelihood
        param X(nd array): input dataset
        """
        gammas = 0
        for k in range(self.k):
            gammas+=np.log((self.pis[k]*multivariate_normal.pdf(X,self.means[k,:],self.covs[k],allow_singular=True)))
        return np.sum(gammas)
        


## Testing

In [201]:
from scipy.stats import multivariate_normal
import numpy as np 

In [202]:
mean1 = np.array([0,0])
cov1 = np.array([[1,-0.8],[-0.8,1]])

mean2 = np.array([1,2.5])
cov2 = np.array([[1,-0.8],[-0.8,1]])

mean3 = np.array([1,5])
cov3 = np.array([[1,-0.8],[-0.8,1]])

n = 300 
np.random.seed(42)

x1, y1 = np.random.multivariate_normal(mean1, cov1, n).T
x2, y2 = np.random.multivariate_normal(mean2, cov2, n).T
x3, y3 = np.random.multivariate_normal(mean3, cov3, n).T

x = np.concatenate((x1,x2,x3)).reshape(-1,1)
y = np.concatenate((y1,y2,y3)).reshape(-1,1)
X = np.concatenate((x,y),axis=1)


In [203]:
from sklearn import datasets

In [204]:
iris = datasets.load_iris(return_X_y=True)

In [205]:
X = iris[0]
y = np.squeeze(iris[1].reshape(-1,1), axis=1) 

In [206]:
model = GMM(3)

In [207]:
model.fit(X)

[[0.33333333]
 [0.33333333]
 [0.33333333]] [[0. 0. 0. 0.]
 [0. 0. 0. 0.]
 [0. 0. 0. 0.]]
[[0.33333333 0.33333333 0.33333333]
 [0.33333333 0.33333333 0.33333333]
 [0.33333333 0.33333333 0.33333333]
 [0.33333333 0.33333333 0.33333333]
 [0.33333333 0.33333333 0.33333333]
 [0.33333333 0.33333333 0.33333333]
 [0.33333333 0.33333333 0.33333333]
 [0.33333333 0.33333333 0.33333333]
 [0.33333333 0.33333333 0.33333333]
 [0.33333333 0.33333333 0.33333333]
 [0.33333333 0.33333333 0.33333333]
 [0.33333333 0.33333333 0.33333333]
 [0.33333333 0.33333333 0.33333333]
 [0.33333333 0.33333333 0.33333333]
 [0.33333333 0.33333333 0.33333333]
 [0.33333333 0.33333333 0.33333333]
 [0.33333333 0.33333333 0.33333333]
 [0.33333333 0.33333333 0.33333333]
 [0.33333333 0.33333333 0.33333333]
 [0.33333333 0.33333333 0.33333333]
 [0.33333333 0.33333333 0.33333333]
 [0.33333333 0.33333333 0.33333333]
 [0.33333333 0.33333333 0.33333333]
 [0.33333333 0.33333333 0.33333333]
 [0.33333333 0.33333333 0.33333333]
 [0.3333333

In [134]:
model.predict(X)

array([[3.93720647e-06, 3.93720647e-06, 3.93720647e-06],
       [3.93720644e-06, 3.93720644e-06, 3.93720644e-06],
       [3.93720643e-06, 3.93720643e-06, 3.93720643e-06],
       [3.93720643e-06, 3.93720643e-06, 3.93720643e-06],
       [3.93720647e-06, 3.93720647e-06, 3.93720647e-06],
       [3.93720651e-06, 3.93720651e-06, 3.93720651e-06],
       [3.93720645e-06, 3.93720645e-06, 3.93720645e-06],
       [3.93720646e-06, 3.93720646e-06, 3.93720646e-06],
       [3.93720642e-06, 3.93720642e-06, 3.93720642e-06],
       [3.93720644e-06, 3.93720644e-06, 3.93720644e-06],
       [3.93720649e-06, 3.93720649e-06, 3.93720649e-06],
       [3.93720646e-06, 3.93720646e-06, 3.93720646e-06],
       [3.93720643e-06, 3.93720643e-06, 3.93720643e-06],
       [3.93720640e-06, 3.93720640e-06, 3.93720640e-06],
       [3.93720651e-06, 3.93720651e-06, 3.93720651e-06],
       [3.93720654e-06, 3.93720654e-06, 3.93720654e-06],
       [3.93720650e-06, 3.93720650e-06, 3.93720650e-06],
       [3.93720647e-06, 3.93720